# User Interface
The user interface is expected to be a must-have for the framework. The original framework did not
come with a user interface and even when was added later it was more focused on the test running
aspect of the framework. 

The intention here is to put both requirements and tests at the forefront. Requirements being
first class enables an experience where tests can be more tightly connected to requirements.


In [ ]:
#export
from flask import render_template, render_template_string, Blueprint
import re
from functools import reduce

import nbloader
nbloader.install_loader()

from nbdev import patch

utils = __import__('Appendix - B - Utilities')

app = Blueprint('app', __name__)

In [ ]:
# Setup the jupyter specifics
from IPython.display import display, HTML, IFrame

jupyter_flask = utils.JupyterFlask(name='automationv2', port=8846, 
                                   static_url_path="/static", static_folder='public')

app = utils.JupyterBlueprint(jupyter_flask, app)

jupyter_flask.register_blueprint(app, url_prefix=None)

## Layout
The application will have a 2 column layout

In [ ]:
sample_template = '''
{% extends "layout.html" %}

{% block menucontent %}
<ul>
    <li>Link 1</li>
    <li>Link 2</li>
</ul>
{% endblock %}

{% block content %}
<h1>MAIN CONTENT</h1>
{% endblock %}
'''

@app.route('/sample')
def sample():
    return render_template_string(sample_template)


In [ ]:
display(IFrame("http://localhost:8846/sample", width=800, height=400))

In [ ]:
@app.route('/test1')
def test1():
    template = '''
    {% extends "layout.html" %}
    
    {% block content %}
        <div hx-get="/test2"><h2>WELCOME HTMX</h2></div>
    {% endblock %}
    '''
    return render_template_string(template)

@app.route('/test2')
def test2():
    return b'<h1>TESTING</h1>'

In [ ]:
#import http.client

#conn = http.client.HTTPConnection("localhost:8846")
#conn.request("GET", "/static2/js/htmx/htmx.min.js")
#conn.request("GET", "/test1")
#res = conn.getresponse()

#html = res.read().decode('utf-8')
display(IFrame("http://localhost:8846/test1", width=800, height=400))
#print(html)